In [5]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torchvision.models as models
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import torchvision.models as models
import os
import pandas as pd
import cv2
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantVillage(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/PlantVillage/PlantVillage-Dataset-master/All/resized512/'
       # self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([transforms.Resize((342,512)),
        transforms.Normalize((124.4455, 159.9584, 104.1832), (47.1528, 41.4626, 49.2424))])
            image = self.transform(image)
            label = self.img_labels['disease'][idx]
            label=np.fromstring(label[1:-1], dtype=np.int,sep=',')
            return image, label

In [7]:
df=pd.read_csv('./plantVillage.csv')
df=df.drop('Unnamed: 0',axis=1)
dataset=plantVillage(df=df)
loader=DataLoader(dataset)

In [10]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through imagesworkbench.action.openSettings
for inputs in (loader):
    psum    += inputs[0].sum(axis        = [0, 2, 3])
    psum_sq += (inputs[0] ** 2).sum(axis = [0, 2, 3])


C:\Users\nonom\AppData\Local\Temp\ipykernel_17796\161376172.py:23: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  label=np.fromstring(label[1:-1], dtype=np.int,sep=',')
C:\Users\nonom\AppData\Local\Temp\ipykernel_17796\161376172.py:23: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  label=np.fromstring(label[1:-1], dtype=np.int,sep=',')


In [23]:
#There are images not in (256,256), lets count these
dir='../../Data/PlantVillage/PlantVillage-Dataset-master/All/resized512/'
for filename in os.listdir(dir):
    temp=read_image(dir+filename)
    if(str(temp.shape)!='torch.Size([3, 256, 256])'):
        print(temp.shape)


torch.Size([3, 256, 167])
torch.Size([3, 256, 235])
torch.Size([3, 256, 233])
torch.Size([3, 256, 162])


In [27]:
# pixel count
count256=4

count = (len(dataset)-count256) * 256 * 256*3
print(count)
count+=256*167*3
count+=256*235*3
print(count)
count+=256*233*3
count+=256*162*3

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

21352611840
21352920576
mean: tensor([-0.7898, -1.4373, -0.6051])
std:  tensor([1.3209, 1.5867, 1.1863])


In [25]:
(psum_sq / count)

tensor([ 7.1061, 13.7505,  5.3205])

In [17]:
(total_mean)**2

tensor([ 5.6143, 18.5933,  3.2956])

In [18]:
total_var

tensor([ 1.4918, -4.8429,  2.0249])

In [ ]:
mean: tensor([125.1284, 160.8532, 104.8010])
std:  tensor([46.3414, 39.7286, 48.7497])

360000000